In [1]:
globals().clear
import time
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
from datetime import datetime

In [2]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

In [3]:
# Load Dataset
df = pd.read_csv('../../df.csv')

In [4]:
# list of tickers for stocks in our data set. Sort the ticker list since our df will be alphabetically arranged.
tickers = ["AAPL", 'XOM', 'IBM', 'KO', 'CVX', 'BA', 'PFE', 'MSFT', 'T', 'WMT',
       'F', 'NFLX', 'JPM', 'MCD', 'GE', 'NVDA', 'JNJ', 'BAC', 'C', 'AMZN',
       'INTC', 'CSCO', 'TSLA', 'GOOGL', 'AMD', 'BABA', 'VZ', 'DIS',
       'META']
tickers.sort()

In [5]:
# For this specific trial drop PLTR since data is not complete
df = df[df.stock_ID != "PLTR"]

In [6]:
df['DATETIME']= pd.to_datetime(df['DATETIME'], format='%m/%d/%Y %H:%M')

In [7]:
# Sort by time so that the first 29 rows occupy the first time value for all the stocks. 
# Fill in 0's for missing values for now.
df = df.set_index('DATETIME')
df = df.fillna(0)

In [8]:
# Select the length of the df ; For this file we need 16 months
# We use the first 12 months to make the first prediction, then shift window
# Then we will repeat this for the next 4 months (Hence total is 16 months needed)
df = df.sort_index().loc['2021-09-01':'2022-12-30']

In [9]:
#Set the DATETIME for fecha 
DATETIME = df.index.values[::29]

In [10]:
# Sort dataframe such that it is both in sequential order, and also in alphabetical order for each day 
# (i.e first entry for each time entry should be AAPL, and last should be XOM).
df.sort_values(["DATETIME", "stock_ID"], inplace=True)

In [11]:
# Select individual stocks and remove demographics
aapl = df.iloc[::29  ,:74]
f    = df.iloc[10::29,:74]
nvda = df.iloc[22::29,:74]
tsla = df.iloc[25::29,:74]
wmt  = df.iloc[27::29,:74]

In [12]:
num_companies = 1                # 1 companies in our dataset.
days = 5
step_rows  = 24 * num_companies * days  # 24 time periods per day per stock
total_rows = len(aapl['2021-09-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(aapl['2021-09-01':'2022-08-31'])  # Define length of training window

scale_X = MinMaxScaler()
model_1 = RandomForestRegressor(n_estimators = 300, max_depth = 100, n_jobs=-1)

aapl_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = aapl.iloc[i:train_rows + i, 1:] 
    test   = aapl.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_1.fit(X_train, y_train)
    y_hat = model_1.predict(X_test)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    aapl_result = pd.concat([aapl_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Count Down: 16
Count Down: 15
Count Down: 14
Count Down: 13
Count Down: 12
Count Down: 11
Count Down: 10
Count Down: 9
Count Down: 8
Count Down: 7
Count Down: 6
Count Down: 5
Count Down: 4
Count Down: 3
Count Down: 2
Count Down: 1
Count Down: 0


In [13]:
num_companies = 1                # 1 companies in our dataset.
days = 5
step_rows  = 24 * num_companies * days  # 24 time periods per day per stock
total_rows = len(f['2021-09-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(f['2021-09-01':'2022-08-31'])  # Define length of training window

scale_X = MinMaxScaler()
model_2 = RandomForestRegressor(n_estimators = 300, max_depth = 100, n_jobs=-1)

f_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = f.iloc[i:train_rows + i, 1:] 
    test   = f.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]
    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_2.fit(X_train, y_train)
    y_hat = model_2.predict(X_test)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    f_result = pd.concat([f_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Count Down: 16
Count Down: 15
Count Down: 14
Count Down: 13
Count Down: 12
Count Down: 11
Count Down: 10
Count Down: 9
Count Down: 8
Count Down: 7
Count Down: 6
Count Down: 5
Count Down: 4
Count Down: 3
Count Down: 2
Count Down: 1
Count Down: 0


In [14]:
num_companies = 1                # 1 companies in our dataset.
days = 5
step_rows  = 24 * num_companies * days  # 24 time periods per day per stock
total_rows = len(nvda['2021-09-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(nvda['2021-09-01':'2022-08-31'])  # Define length of training window

scale_X = MinMaxScaler()
model_3 = RandomForestRegressor(n_estimators = 300, max_depth = 100, n_jobs=-1)

nvda_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = nvda.iloc[i:train_rows + i, 1:] 
    test   = nvda.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]
    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_3.fit(X_train, y_train)
    y_hat = model_3.predict(X_test)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    nvda_result = pd.concat([nvda_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Count Down: 16
Count Down: 15
Count Down: 14
Count Down: 13
Count Down: 12
Count Down: 11
Count Down: 10
Count Down: 9
Count Down: 8
Count Down: 7
Count Down: 6
Count Down: 5
Count Down: 4
Count Down: 3
Count Down: 2
Count Down: 1
Count Down: 0


In [15]:
num_companies = 1                # 1 companies in our dataset.
days = 5
step_rows  = 24 * num_companies * days  # 24 time periods per day per stock
total_rows = len(tsla['2021-09-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(tsla['2021-09-01':'2022-08-31'])  # Define length of training window

scale_X = MinMaxScaler()
model_4 = RandomForestRegressor(n_estimators = 300, max_depth = 100, n_jobs=-1)

tsla_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = tsla.iloc[i:train_rows + i, 1:] 
    test   = tsla.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]
    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_4.fit(X_train, y_train)
    y_hat = model_4.predict(X_test)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    tsla_result = pd.concat([tsla_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Count Down: 16
Count Down: 15
Count Down: 14
Count Down: 13
Count Down: 12
Count Down: 11
Count Down: 10
Count Down: 9
Count Down: 8
Count Down: 7
Count Down: 6
Count Down: 5
Count Down: 4
Count Down: 3
Count Down: 2
Count Down: 1
Count Down: 0


In [16]:
num_companies = 1                # 1 companies in our dataset.
days = 5
step_rows  = 24 * num_companies * days  # 24 time periods per day per stock
total_rows = len(wmt['2021-09-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(wmt['2021-09-01':'2022-08-31'])  # Define length of training window

scale_X = MinMaxScaler()
model_5 = RandomForestRegressor(n_estimators = 300, max_depth = 100, n_jobs=-1)

wmt_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = wmt.iloc[i:train_rows + i, 1:] 
    test   = wmt.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]
    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_5.fit(X_train, y_train)
    y_hat = model_5.predict(X_test)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    wmt_result = pd.concat([wmt_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Count Down: 16
Count Down: 15
Count Down: 14
Count Down: 13
Count Down: 12
Count Down: 11
Count Down: 10
Count Down: 9
Count Down: 8
Count Down: 7
Count Down: 6
Count Down: 5
Count Down: 4
Count Down: 3
Count Down: 2
Count Down: 1
Count Down: 0


In [17]:
metrics_df = pd.DataFrame(columns = ['stock_ID', "RMSE", "MAPE", "MPE", "MTT"])

In [18]:
def mean_positive_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    error = np.mean(np.maximum((y_pred - y_true),0))
    return error

In [19]:
new_rows = []
stock_result = aapl_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['AAPL',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,AAPL,3.994475,2.086039,2.240701,6.321083


In [20]:
new_rows = []
stock_result = f_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['F',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,F,0.857277,4.678358,0.411556,6.089034


In [21]:
new_rows = []
stock_result = nvda_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['NVDA',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,NVDA,10.642683,5.481911,3.653528,6.320752


In [22]:
new_rows = []
stock_result = tsla_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['TSLA',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,TSLA,22.712344,8.855921,13.998805,6.351661


In [23]:
new_rows = []
stock_result = wmt_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['WMT',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,WMT,3.454106,1.901445,0.836146,6.142909
